In [2]:
import ROOT
import pythia8
from tqdm import tqdm
import numpy as np

In [3]:
# Output file and TTree
filename = "pt10_10000.root"
ptHard = 10
f = ROOT.TFile(filename, "RECREATE")
tree = ROOT.TTree("EventTree", "Pythia Events with TParticles")

# TClonesArray of TParticle - needed to fill the tree, 1000 is minimum entries, but it can add more by default
particles = ROOT.TClonesArray("TParticle", 1000)

# branches of the tree are arrays of TParticles
tree.Branch("Particles", particles)

# Set up Pythia
pythia = pythia8.Pythia()
pythia.readString("HardQCD:all = on")
pythia.readString(f"PhaseSpace:pTHatMin = {ptHard}")
pythia.readString("Beams:idA = 2212")
pythia.readString("Beams:idB = 2212")
pythia.readString("Beams:eCM = 13000.")
pythia.readString("Random:setSeed = on")
pythia.readString("Random:seed = 4")
pythia.init()

# Event loop
for iEvent in tqdm(range(10000)):
    if not pythia.next():
        continue

    # clears the array from the previous loop
    particles.Clear()           

    for i in range(pythia.event.size()):
        p = pythia.event[i]

        mom = ROOT.TLorentzVector(p.px(), p.py(), p.pz(), p.e())
        prod = ROOT.TLorentzVector(p.xProd(), p.yProd(), p.zProd(), p.tProd())

        # create TParticle object for the current pythia particle
        particle = ROOT.TParticle(      
            p.id(), p.statusHepMC(),
            p.mother1(), p.mother2(),
            p.daughter1(), p.daughter2(),
            mom, prod
        )

        # assigns the TParticle ith index of the array 
        particles.ConstructedAt(i).__assign__(particle)       

    # fills a branch of the tree with the TParticle array
    tree.Fill()

f.Write()
f.Close()

100%|██████████| 10000/10000 [05:09<00:00, 32.34it/s]



 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.312      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 23 May 2024  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 18 Apr 2025 at 13:03:41    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

Warning in <TTree::Bronch>: Using split mode on a class: TParticle with a custom Streamer
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 90 unknown from TDatabasePDG


Here, I created a loop which generated 1000 event files using various ptHard values.

In [ ]:
# Define pTHatMin values
pt_hat_values = np.linspace(0, 20, 11)  # 0 to 20 GeV in 10 steps

for pt_hat in tqdm(pt_hat_values, desc="Generating files for different pTHatMin"):
    # Output file and TTree
    filename = f"genpt{pt_hat:.1f}.root"
    f = ROOT.TFile(filename, "RECREATE")
    tree = ROOT.TTree("EventTree", "Pythia Events with TParticles")

    # TClonesArray of TParticle
    particles = ROOT.TClonesArray("TParticle", 1000)
    tree.Branch("Particles", particles)

    # Set up Pythia
    pythia = pythia8.Pythia()
    pythia.readString("HardQCD:all = on")
    pythia.readString(f"PhaseSpace:pTHatMin = {pt_hat}")
    pythia.readString("Beams:idA = 2212")
    pythia.readString("Beams:idB = 2212")
    pythia.readString("Beams:eCM = 13000.")
    pythia.readString("Random:setSeed = on")
    pythia.readString("Random:seed = 4")
    pythia.init()

    # Event loop
    for iEvent in range(1000):
        if not pythia.next():
            continue

        particles.Clear()

        for i in range(pythia.event.size()):
            p = pythia.event[i]

            mom = ROOT.TLorentzVector(p.px(), p.py(), p.pz(), p.e())
            prod = ROOT.TLorentzVector(p.xProd(), p.yProd(), p.zProd(), p.tProd())

            particle = ROOT.TParticle(
                p.id(), p.statusHepMC(),
                p.mother1(), p.mother2(),
                p.daughter1(), p.daughter2(),
                mom, prod
            )

            particles.ConstructedAt(i).__assign__(particle)

        tree.Fill()

    f.Write()
    f.Close()

print("Generation complete! Files saved as genpt{value}.root")
print("Generated files:")
for pt_hat in pt_hat_values:
    print(f"genpt{pt_hat:.1f}.root")